# 🎓 Fine-Tuning LLMs with LoRA on Apple Silicon

## What You'll Learn
- **LoRA (Low-Rank Adaptation)**: Train a 1B parameter model on a laptop
- **Quantization**: Run models 4x smaller with 4-bit precision
- **MLX-LM**: Apple's unified interface for LLMs
- **Production Ready**: Chat templates and modern best practices (2025)

## The Problem: Traditional Fine-Tuning is Expensive

Training a 7B parameter model requires:
- **Full Fine-Tuning**: 80GB+ VRAM, $10,000+ GPU
- **LoRA**: 16GB RAM, Your MacBook (Free!)

## The Solution: LoRA

Instead of updating all 7 billion parameters, LoRA:
1. **Freezes** the original model weights
2. **Adds** small "adapter" matrices (rank-deficient)
3. **Trains** only these adapters (<1% of parameters)

Result: 99% of the performance, 1% of the memory.

---

In [1]:
# Install mlx-lm if not already installed
!pip install mlx-lm

In [2]:
import sys
from mlx_lm import load, generate
from mlx_nlp_utils import print_device_info

print_device_info()


🖥️  Hardware Acceleration Check:
   Device: Device(gpu, 0)
   ✅ Using Apple Silicon GPU (Metal)
   ℹ️  MLX automatically optimizes for the GPU's Unified Memory.
   ℹ️  Note: While Apple Silicon has an NPU (Neural Engine), MLX primarily
       uses the powerful GPU for general-purpose training tasks like LSTMs.


## 1. Load a Pre-Trained Model

We will use a small but capable model like **Mistral-7B** or **TinyLlama** (depending on your RAM). MLX handles the downloading automatically from Hugging Face.

In [3]:
# Load model and tokenizer
# Using Llama-3.2-1B-Instruct (State of the art small model as of late 2024/2025)
model_name = "mlx-community/Llama-3.2-1B-Instruct-4bit"
print(f"Loading {model_name}...")

model, tokenizer = load(model_name)

print("✅ Model loaded!")

Loading mlx-community/Llama-3.2-1B-Instruct-4bit...


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Model loaded!


## 2. Test Base Model

Let's see how it performs *before* fine-tuning.

In [4]:
# Use the tokenizer's chat template (Modern Best Practice)
messages = [
    {"role": "user", "content": "What is the capital of France?"}
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(f"Formatted Prompt:\n{prompt}\n{'='*20}")

response = generate(model, tokenizer, prompt=prompt, verbose=True)

Formatted Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 19 Nov 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


The capital capital of France of France is Paris is Paris..
Prompt: 42 tokens, 447.080 tokens-per-sec
Generation: 8 tokens, 144.286 tokens-per-sec
Peak memory: 0.755 GB

Prompt: 42 tokens, 447.080 tokens-per-sec
Generation: 8 tokens, 144.286 tokens-per-sec
Peak memory: 0.755 GB


## 3. Prepare Data for Fine-Tuning

We need data in a specific format (JSONL) for `mlx-lm`. Let's convert our Intent/Sentiment data.

In [5]:
import json
from pathlib import Path

# 1. Load our existing Intent Classification data
data_path = Path('../data/intent_samples/data.json')

if not data_path.exists():
    print("⚠️ Data not found. Please run: python ../scripts/download_datasets.py --samples")
else:
    with open(data_path, 'r') as f:
        raw_data = json.load(f)
    
    texts = raw_data['texts']
    labels = raw_data['labels']
    
    # 2. Convert to Chat Format (JSONL)
    # We want the model to learn to classify intents.
    # User: "Turn on the lights" -> Assistant: "intent: command"
    
    chat_data = []
    for text, label in zip(texts, labels):
        entry = {
            "messages": [
                {"role": "user", "content": f"Classify the intent of this text: '{text}'"},
                {"role": "assistant", "content": f"intent: {label}"}
            ]
        }
        chat_data.append(entry)
    
    # 3. Save as train.jsonl and valid.jsonl
    # Split 80/20
    split_idx = int(len(chat_data) * 0.8)
    train_data = chat_data[:split_idx]
    valid_data = chat_data[split_idx:]
    
    # Ensure data directory exists
    Path('data').mkdir(exist_ok=True)
    
    def save_jsonl(data, filename):
        with open(filename, 'w') as f:
            for entry in data:
                json.dump(entry, f)
                f.write('\n')
        print(f"Saved {len(data)} examples to {filename}")

    save_jsonl(train_data, 'data/train.jsonl')
    save_jsonl(valid_data, 'data/valid.jsonl')
    
    print("\n✅ Data prepared for MLX LoRA!")
    print("   Format: JSONL (Chat format)")

Saved 7 examples to data/train.jsonl
Saved 2 examples to data/valid.jsonl

✅ Data prepared for MLX LoRA!
   Format: JSONL (Chat format)


## 4. Run Fine-Tuning (LoRA)

We can use the `mlx_lm.lora` command line tool or API to train.

In [6]:
# Example command to run fine-tuning
# We use the CLI tool provided by mlx-lm
# --batch-size 4: Fits easily on 8GB/16GB RAM
# --lora-layers 16: Target more layers for better quality
# --iters 600: Enough for a small dataset

cmd = """
python -m mlx_lm.lora \\
    --model mlx-community/Llama-3.2-1B-Instruct-4bit \\
    --train \\
    --data data \\
    --batch-size 4 \\
    --lora-layers 16 \\
    --iters 600 \\
    --learning-rate 1e-4
"""

print("Run this command in your terminal:")
print(cmd)

# Note: You can also run it directly here with !python ...
# !{cmd}

Run this command in your terminal:

python -m mlx_lm.lora \
    --model mlx-community/Llama-3.2-1B-Instruct-4bit \
    --train \
    --data data \
    --batch-size 4 \
    --lora-layers 16 \
    --iters 600 \
    --learning-rate 1e-4



In [10]:
# Option: Run training directly in notebook
# Uncomment to train (this will take 5-10 minutes)

# import subprocess
# result = subprocess.run([
#     "python", "-m", "mlx_lm.lora",
#     "--model", "mlx-community/Llama-3.2-1B-Instruct-4bit",
#     "--train",
#     "--data", "data",
#     "--batch-size", "4",
#     "--lora-layers", "16", 
#     "--iters", "600",
#     "--learning-rate", "1e-4",
#     "--adapter-path", "./adapters"
# ], capture_output=True, text=True)
# 
# print(result.stdout)
# print(result.stderr)

### What the Training Does

The LoRA training will:
1. Download the base model (if not cached)
2. Load your training/validation data
3. Add LoRA adapters to attention layers
4. Train for 600 iterations (~5-10 minutes on M1/M2)
5. Save the adapters to `./adapters`

**Note**: This is a demo. For production:
- Use more data (100+ examples minimum)
- Train longer (1000-5000 iterations)
- Tune hyperparameters (learning rate, rank, alpha)

## 5. Inference with Fine-Tuned Model

After training completes, we can load the base model + adapters and test it.

In [8]:
# Load the fine-tuned model (base + adapters) if available
from pathlib import Path

adapter_path = Path("./adapters")

if adapter_path.exists():
    print("✅ Loading fine-tuned model (base + LoRA adapters)...")
    model_ft, tokenizer_ft = load(model_name, adapter_path=str(adapter_path))
    print("✅ Fine-tuned model loaded!")
else:
    print("⚠️  No adapters found. Will use base model only.")
    print("   (Run the training command first to create adapters)")
    model_ft, tokenizer_ft = model, tokenizer

# Side-by-side comparison
test_query = "Turn off the lights"

messages = [
    {"role": "user", "content": f"Classify the intent of this text: '{test_query}'"}
]
prompt_test = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(f"\nTest Query: '{test_query}'")
print("="*60)

print("\n📦 BASE MODEL:")
response_base = generate(model, tokenizer, prompt=prompt_test, max_tokens=50, verbose=False)
print(response_base)

if adapter_path.exists():
    print("\n🎯 FINE-TUNED MODEL:")
    response_ft = generate(model_ft, tokenizer_ft, prompt=prompt_test, max_tokens=50, verbose=False)
    print(response_ft)
else:
    print("\n⚠️  Fine-tuned model not available (run training first)")

print("="*60)
print("\n💡 Expected Improvement:")
print("   - Base model: Generic response or incorrect intent")
print("   - Fine-tuned: Correctly identifies 'intent: command'")

⚠️  No adapters found. Will use base model only.
   (Run the training command first to create adapters)

Test Query: 'Turn off the lights'

📦 BASE MODEL:
The intent of this text is to instruct or guide someone to perform a specific action, which in this case is to turn off the lights. It is a direct and clear command, likely from a household or home maintenance context.

⚠️  Fine-tuned model not available (run training first)

💡 Expected Improvement:
   - Base model: Generic response or incorrect intent
   - Fine-tuned: Correctly identifies 'intent: command'
The intent of this text is to instruct or guide someone to perform a specific action, which in this case is to turn off the lights. It is a direct and clear command, likely from a household or home maintenance context.

⚠️  Fine-tuned model not available (run training first)

💡 Expected Improvement:
   - Base model: Generic response or incorrect intent
   - Fine-tuned: Correctly identifies 'intent: command'


## 6. Compare: Base vs Fine-Tuned

Let's see the difference between the base model and our fine-tuned version.

In [9]:
# Test the fine-tuned model on intent classification
if not adapter_path.exists():
    print("⚠️  No fine-tuned model available. Run training first.")
    print("   Using base model for testing.")

test_cases = [
    "Turn on the light",
    "What time is it",
    "Hello there",
    "Set a timer for 5 minutes",
    "How are you doing"
]

model_to_test = model_ft if adapter_path.exists() else model
tokenizer_to_test = tokenizer_ft if adapter_path.exists() else tokenizer
model_type = "Fine-Tuned" if adapter_path.exists() else "Base"

print(f"\n🧪 Testing {model_type} Model\n" + "="*50)

for test_text in test_cases:
    messages = [
        {"role": "user", "content": f"Classify the intent of this text: '{test_text}'"}
    ]
    
    prompt = tokenizer_to_test.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    print(f"\n📝 Input: '{test_text}'")
    response = generate(model_to_test, tokenizer_to_test, prompt=prompt, max_tokens=50, verbose=False)
    print(f"🤖 Output: {response}")

print("\n" + "="*50)

⚠️  No fine-tuned model available. Run training first.
   Using base model for testing.

🧪 Testing Base Model

📝 Input: 'Turn on the light'
🤖 Output: The intent of this text is **Instructive** or **Practical**. It provides a clear and direct instruction on how to perform a simple task (turning on a light), which is likely intended to be helpful or useful for someone who

📝 Input: 'What time is it'
🤖 Output: The intent of this text is **Instructive** or **Practical**. It provides a clear and direct instruction on how to perform a simple task (turning on a light), which is likely intended to be helpful or useful for someone who

📝 Input: 'What time is it'
🤖 Output: The intent of this text is to **inform** or **convey information**. Specifically, it appears to be a simple, factual statement asking the reader to provide the current time, which is a common way to ask for the time in everyday conversation

📝 Input: 'Hello there'
🤖 Output: The intent of this text is to **inform** or **convey 

## 🎉 Conclusion

You have now completed the journey from simple RNNs to fine-tuning modern LLMs on Apple Silicon!

**What's Next?**
- Build a RAG (Retrieval Augmented Generation) system.
- Deploy your fine-tuned model as an API.

## ❓ FAQ

**Q: LoRA vs. Full Fine-Tuning?**
A:
*   **Full Fine-Tuning:** Updates all weights. Requires massive VRAM (e.g., 80GB+ for 7B model).
*   **LoRA:** Updates <1% of weights. Runs on consumer hardware (e.g., 16GB MacBook). Performance is often 98-99% of full fine-tuning.

**Q: Can I fine-tune on my own emails?**
A: Yes! Just format them as JSONL: `{"messages": [{"role": "user", "content": "Subject: Meeting"}, {"role": "assistant", "content": "Hi team..."}]}`.

**Q: What is "Quantization" (4-bit)?**
A: It reduces the precision of weights from 16-bit (Float16) to 4-bit integers. This cuts memory usage by 4x with minimal loss in quality, allowing you to run a 7B model on a laptop.

## 💭 Closing Thoughts

**The Commoditization of Intelligence**
We are entering an era where "Intelligence" is a downloadable package.
*   **2020:** Only OpenAI had GPT-3.
*   **2025:** You can run a model nearly as smart as GPT-3 on your laptop, fine-tuned on your private data, with no internet connection.

**Architectural Shift:**
The role of the AI Engineer is shifting from "Designing Architectures" (building LSTMs) to "Data Curation" (preparing high-quality datasets for fine-tuning). The model is a solved problem; the data is your competitive advantage.